In [1]:
import cv2
import numpy as np
import os
#import time
import mediapipe as mp
import pandas as pd
# import matplotlib.pyplot as plt

In [2]:
if not os.path.exists('./image'):
    os.mkdir('./image')

In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [7]:
# create dataframe base
lv1 = [x.name for x in list(mp_pose.PoseLandmark)]
lv2 = ['x', 'y', 'z', 'v']
columns = pd.MultiIndex.from_product([lv1, lv2]).insert(0, ('playtime', 'sec'))
df = pd.DataFrame(columns=columns)

cap = cv2.VideoCapture("./dtw_sample_train.mp4")

# video info
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000/fps)
print("height :", cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("width :", cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print("fps :", fps)

#cnt = -1

# make mediapipe pose model
with mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    smooth_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5,) as pose:
    
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        if not ret:
            break
        
        #playtime = round(cap.get(cv2.CAP_PROP_POS_MSEC)/1000, 2)
        
        # make skeleton image
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        results = pose.process(img)
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            #print(landmarks)
        except:
            pass

        #mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        #cv2.imshow("test", img)
        
        #cnt += 1
        
        # save image & skeleton data
        #if cv2.waitKey(delay) & 0xFF == ord('c'):
        #if cnt % 30 == 0:
        #cv2.imwrite(f"./image/{playtime}.jpg", img) #frame : 골격x / img : 골격o
        xyzv = np.array([[res.x, res.y, res.z, res.visibility]for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
        xyzv = np.insert(xyzv, 0, cap.get(cv2.CAP_PROP_POS_FRAMES))
        df = pd.concat([df, pd.DataFrame([xyzv], columns=columns)], ignore_index=True)
        #continue
            
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    df.to_csv("./image/dtw_sample_train_skeleton.csv", index=False) # save skeleton data

height : 720.0
width : 640.0
fps : 29.97002997002997


In [ ]:
cap.release()
cv2.destroyAllWindows()

# load test, train data

In [3]:
test = pd.read_csv("./image/dtw_sample_test_skeleton.csv", header=[0,1])

In [4]:
train = pd.read_csv("./image/dtw_sample_train_skeleton.csv", header=[0,1])

In [93]:
test_d = test.drop(test.loc[:,(slice(None), ['z','v'])], axis=1)

In [94]:
test_d.head()

playtime      NOSE           LEFT_EYE_INNER            LEFT_EYE            \
       sec         x         y              x         y         x         y   
0      1.0  0.517440  0.191790       0.525520  0.180241  0.531481  0.180967   
1      2.0  0.515312  0.191814       0.524741  0.180255  0.530590  0.180986   
2      3.0  0.511461  0.192231       0.522251  0.180483  0.528537  0.181172   
3      4.0  0.509108  0.193643       0.518850  0.181364  0.525608  0.181895   
4      5.0  0.508706  0.197692       0.517989  0.185085  0.524685  0.185464   

  LEFT_EYE_OUTER           RIGHT_EYE_INNER  ... RIGHT_ANKLE            \
               x         y               x  ...           x         y   
0       0.538088  0.182000        0.506933  ...    0.464793  0.796986   
1       0.536555  0.182032        0.505676  ...    0.465125  0.798281   
2       0.534004  0.182185        0.502326  ...    0.465270  0.798566   
3       0.530920  0.182657        0.498691  ...    0.465648  0.798527   
4       0.529591  0.185836        0.497849  ...    0.465786  0.799773   

  LEFT_HEEL           RIGHT_HEEL           LEFT_FOOT_INDEX            \
          x         y          x         y               x         y   
0  0.555255  0.839984   0.470715  0.820219        0.588020  0.871516   
1  0.555530  0.837718   0.471077  0.821383        0.587197  0.869676   
2  0.555894  0.834878   0.471470  0.822034        0.587022  0.865889   
3  0.556102  0.830296   0.472023  0.821977        0.585917  0.860315   
4  0.556113  0.827486   0.472339  0.824214        0.585518  0.854297   

  RIGHT_FOOT_INDEX            
                 x         y  
0         0.435271  0.850148  
1         0.434729  0.851558  
2         0.432286  0.852560  
3         0.433114  0.851815  
4         0.433030  0.851337  

[5 rows x 67 columns]

In [97]:
train_d = train.drop(train.loc[:,(slice(None),['z','v'])], axis=1)

In [98]:
train_d.head()

playtime      NOSE           LEFT_EYE_INNER            LEFT_EYE            \
       sec         x         y              x         y         x         y   
0      1.0  0.530722  0.288515       0.536021  0.274355  0.540506  0.274250   
1      2.0  0.531541  0.288517       0.536534  0.274353  0.540921  0.274250   
2      3.0  0.531779  0.286608       0.536731  0.272949  0.541114  0.272977   
3      4.0  0.532713  0.285407       0.537325  0.271886  0.541724  0.271931   
4      5.0  0.533324  0.283762       0.537871  0.270842  0.542315  0.270960   

  LEFT_EYE_OUTER           RIGHT_EYE_INNER  ... RIGHT_ANKLE            \
               x         y               x  ...           x         y   
0       0.545191  0.274156        0.518991  ...    0.511483  0.784886   
1       0.545709  0.274160        0.519665  ...    0.511481  0.782006   
2       0.545953  0.273080        0.519876  ...    0.511337  0.781147   
3       0.546528  0.272094        0.520972  ...    0.511129  0.780958   
4       0.547047  0.271220        0.521723  ...    0.510768  0.780843   

  LEFT_HEEL           RIGHT_HEEL           LEFT_FOOT_INDEX            \
          x         y          x         y               x         y   
0  0.562543  0.793876   0.519781  0.798034        0.563856  0.829309   
1  0.562445  0.789262   0.519233  0.790564        0.564073  0.828517   
2  0.562497  0.787841   0.518558  0.789669        0.564106  0.827820   
3  0.562530  0.787342   0.517924  0.789782        0.564040  0.827544   
4  0.562514  0.787462   0.517047  0.789852        0.563863  0.827439   

  RIGHT_FOOT_INDEX            
                 x         y  
0         0.507596  0.830114  
1         0.507779  0.830136  
2         0.507773  0.830136  
3         0.507768  0.830144  
4         0.507735  0.830162  

[5 rows x 67 columns]

# extract x, y from skeleton

In [7]:
test_xy = test[[(a,b )for a, b in test.columns if b in ['x', 'y']]]

In [8]:
train_xy = train[[(a,b) for a, b in train.columns if b in ['x','y']]]

In [9]:
test_xy.head()

NOSE           LEFT_EYE_INNER            LEFT_EYE            \
          x         y              x         y         x         y   
0  0.517445  0.191793       0.525521  0.180241  0.531482  0.180966   
1  0.515302  0.191817       0.524739  0.180256  0.530589  0.180986   
2  0.511461  0.192224       0.522258  0.180479  0.528541  0.181168   
3  0.509118  0.193631       0.518869  0.181356  0.525626  0.181888   
4  0.508741  0.197686       0.518027  0.185071  0.524716  0.185452   

  LEFT_EYE_OUTER           RIGHT_EYE_INNER            ... RIGHT_ANKLE  \
               x         y               x         y  ...           x   
0       0.538088  0.181998        0.506935  0.179597  ...    0.464792   
1       0.536554  0.182031        0.505668  0.179659  ...    0.465120   
2       0.534010  0.182180        0.502332  0.180031  ...    0.465259   
3       0.530935  0.182649        0.498708  0.181279  ...    0.465668   
4       0.529615  0.185826        0.497886  0.184992  ...    0.465799   

            LEFT_HEEL           RIGHT_HEEL           LEFT_FOOT_INDEX  \
          y         x         y          x         y               x   
0  0.796973  0.555264  0.839961   0.470708  0.820200        0.588006   
1  0.798304  0.555535  0.837694   0.471069  0.821400        0.587254   
2  0.798569  0.555912  0.834894   0.471394  0.821823        0.587076   
3  0.798537  0.556120  0.830283   0.471993  0.821794        0.585953   
4  0.799839  0.556121  0.827549   0.472324  0.824196        0.585583   

            RIGHT_FOOT_INDEX            
          y                x         y  
0  0.871499         0.435281  0.850134  
1  0.869666         0.434718  0.851563  
2  0.866018         0.432301  0.852549  
3  0.860405         0.433180  0.851755  
4  0.854249         0.433059  0.851347  

[5 rows x 66 columns]

In [10]:
train_xy.head()

NOSE           LEFT_EYE_INNER            LEFT_EYE            \
          x         y              x         y         x         y   
0  0.530731  0.288516       0.536028  0.274357  0.540514  0.274251   
1  0.531580  0.288518       0.536557  0.274355  0.540944  0.274252   
2  0.531820  0.286597       0.536752  0.272939  0.541135  0.272965   
3  0.532706  0.285426       0.537326  0.271892  0.541725  0.271938   
4  0.533305  0.283768       0.537859  0.270843  0.542301  0.270962   

  LEFT_EYE_OUTER           RIGHT_EYE_INNER            ... RIGHT_ANKLE  \
               x         y               x         y  ...           x   
0       0.545197  0.274158        0.519003  0.275286  ...    0.511478   
1       0.545731  0.274163        0.519707  0.275251  ...    0.511476   
2       0.545973  0.273069        0.519916  0.273627  ...    0.511337   
3       0.546533  0.272102        0.520965  0.272405  ...    0.511134   
4       0.547040  0.271223        0.521696  0.271196  ...    0.510770   

            LEFT_HEEL           RIGHT_HEEL           LEFT_FOOT_INDEX  \
          y         x         y          x         y               x   
0  0.784882  0.562547  0.793881   0.519770  0.798020        0.563854   
1  0.781954  0.562443  0.789182   0.519229  0.790461        0.564074   
2  0.781129  0.562497  0.787886   0.518568  0.789654        0.564104   
3  0.780945  0.562532  0.787322   0.517938  0.789767        0.564044   
4  0.780849  0.562516  0.787457   0.517054  0.789878        0.563867   

            RIGHT_FOOT_INDEX            
          y                x         y  
0  0.829312         0.507605  0.830115  
1  0.828541         0.507796  0.830139  
2  0.827849         0.507788  0.830138  
3  0.827569         0.507785  0.830148  
4  0.827458         0.507750  0.830169  

[5 rows x 66 columns]

# seperate upper and lower body

In [14]:
test_upper = test_xy[[(   'LEFT_SHOULDER', 'x'), (   'LEFT_SHOULDER', 'y'),
                      (  'RIGHT_SHOULDER', 'x'), (  'RIGHT_SHOULDER', 'y'),
                      (      'LEFT_ELBOW', 'x'), (      'LEFT_ELBOW', 'y'),
                      (     'RIGHT_ELBOW', 'x'), (     'RIGHT_ELBOW', 'y'),
                      (      'LEFT_WRIST', 'x'), (      'LEFT_WRIST', 'y'),
                      (     'RIGHT_WRIST', 'x'), (     'RIGHT_WRIST', 'y'),
                      (      'LEFT_PINKY', 'x'), (      'LEFT_PINKY', 'y'),
                      (     'RIGHT_PINKY', 'x'), (     'RIGHT_PINKY', 'y'),
                      (      'LEFT_INDEX', 'x'), (      'LEFT_INDEX', 'y'),
                      (     'RIGHT_INDEX', 'x'), (     'RIGHT_INDEX', 'y'),
                      (      'LEFT_THUMB', 'x'), (      'LEFT_THUMB', 'y'),
                      (     'RIGHT_THUMB', 'x'), (     'RIGHT_THUMB', 'y'),
                      (        'LEFT_HIP', 'x'), (        'LEFT_HIP', 'y'),
                      (       'RIGHT_HIP', 'x'), (       'RIGHT_HIP', 'y')]]

In [15]:
test_lower = test_xy[[(        'LEFT_HIP', 'x'), (        'LEFT_HIP', 'y'),
                      (       'RIGHT_HIP', 'x'), (       'RIGHT_HIP', 'y'),
                      (       'LEFT_KNEE', 'x'), (       'LEFT_KNEE', 'y'),
                      (      'RIGHT_KNEE', 'x'), (      'RIGHT_KNEE', 'y'),
                      (      'LEFT_ANKLE', 'x'), (      'LEFT_ANKLE', 'y'),
                      (     'RIGHT_ANKLE', 'x'), (     'RIGHT_ANKLE', 'y'),
                      (       'LEFT_HEEL', 'x'), (       'LEFT_HEEL', 'y'),
                      (      'RIGHT_HEEL', 'x'), (      'RIGHT_HEEL', 'y'),
                      ( 'LEFT_FOOT_INDEX', 'x'), ( 'LEFT_FOOT_INDEX', 'y'),
                      ('RIGHT_FOOT_INDEX', 'x'), ('RIGHT_FOOT_INDEX', 'y')]]

In [18]:
train_upper = train_xy[[(   'LEFT_SHOULDER', 'x'), (   'LEFT_SHOULDER', 'y'),
                        (  'RIGHT_SHOULDER', 'x'), (  'RIGHT_SHOULDER', 'y'),
                        (      'LEFT_ELBOW', 'x'), (      'LEFT_ELBOW', 'y'),
                        (     'RIGHT_ELBOW', 'x'), (     'RIGHT_ELBOW', 'y'),
                        (      'LEFT_WRIST', 'x'), (      'LEFT_WRIST', 'y'),
                        (     'RIGHT_WRIST', 'x'), (     'RIGHT_WRIST', 'y'),
                        (      'LEFT_PINKY', 'x'), (      'LEFT_PINKY', 'y'),
                        (     'RIGHT_PINKY', 'x'), (     'RIGHT_PINKY', 'y'),
                        (      'LEFT_INDEX', 'x'), (      'LEFT_INDEX', 'y'),
                        (     'RIGHT_INDEX', 'x'), (     'RIGHT_INDEX', 'y'),
                        (      'LEFT_THUMB', 'x'), (      'LEFT_THUMB', 'y'),
                        (     'RIGHT_THUMB', 'x'), (     'RIGHT_THUMB', 'y'),
                        (        'LEFT_HIP', 'x'), (        'LEFT_HIP', 'y'),
                        (       'RIGHT_HIP', 'x'), (       'RIGHT_HIP', 'y')]]

In [19]:
train_lower = train_xy[[(        'LEFT_HIP', 'x'), (        'LEFT_HIP', 'y'),
                        (       'RIGHT_HIP', 'x'), (       'RIGHT_HIP', 'y'),
                        (       'LEFT_KNEE', 'x'), (       'LEFT_KNEE', 'y'),
                        (      'RIGHT_KNEE', 'x'), (      'RIGHT_KNEE', 'y'),
                        (      'LEFT_ANKLE', 'x'), (      'LEFT_ANKLE', 'y'),
                        (     'RIGHT_ANKLE', 'x'), (     'RIGHT_ANKLE', 'y'),
                        (       'LEFT_HEEL', 'x'), (       'LEFT_HEEL', 'y'),
                        (      'RIGHT_HEEL', 'x'), (      'RIGHT_HEEL', 'y'),
                        ( 'LEFT_FOOT_INDEX', 'x'), ( 'LEFT_FOOT_INDEX', 'y'),
                        ('RIGHT_FOOT_INDEX', 'x'), ('RIGHT_FOOT_INDEX', 'y')]]

# flatten data by frame

In [20]:
test_upper_list = test_upper.values

In [21]:
test_lower_list = test_lower.values

In [22]:
train_upper_list = train_upper.values

In [23]:
train_lower_list = train_lower.values

# cosine similarity

In [24]:
def cosine_similarity(x_array, y_array):
    cossim_list = []
    for x, y in zip(x_array, y_array):
        cossim = np.dot(x, y) / (np.linalg.norm(x)*np.linalg.norm(y))
        cossim_list.append(cossim)
    return cossim_list

In [25]:
np.mean(cosine_similarity(test_upper_list, train_upper_list))

0.9800199206525881

In [ ]:
np.mean(cosine_similarity(test_lower_list, train_lower_list))

0.998742351208348

- 상하체 나눠서 체크하면 큰 차이를 보일 것이라고 예상했는데 그렇지 못한 결과가 나옴